In [ ]:
%pip install llama-index
%pip install llama-index-llms-palm
%pip install pypdf
%pip install docx2txt
%pip install google-generativeai
%pip install transformers

# Additional to use Google GenAI Gemini
%pip install llama-index-llms-google-genai
%pip install google-genai
%pip install llama-index-embeddings-google-genai



In [30]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
# Deprecated
#from llama_index.llms.palm import PaLM
from llama_index.llms.google_genai import GoogleGenAI
from llama_index.core import ServiceContext
from llama_index.core import StorageContext, load_index_from_storage

import os
from dotenv import load_dotenv

load_dotenv()
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY

In [49]:
documents = SimpleDirectoryReader("data").load_data()

In [50]:
from google import genai
from llama_index.embeddings.google_genai import GoogleGenAIEmbedding
from llama_index.core import Settings
from llama_index.core.node_parser import SentenceSplitter

Settings.llm = GoogleGenAI(model="gemini-2.0-flash")
Settings.embed_model = GoogleGenAIEmbedding(model_name="gemini-embedding-001")
Settings.node_parser = SentenceSplitter(chunk_size=512, chunk_overlap=20)
Settings.num_output = 512
Settings.context_window = 3900

2025-09-29 22:18:31,080 - INFO - HTTP Request: GET https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash "HTTP/1.1 200 OK"


In [35]:
index = VectorStoreIndex.from_documents(documents)

2025-09-29 22:01:32,371 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-embedding-001:batchEmbedContents "HTTP/1.1 200 OK"
2025-09-29 22:01:32,765 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-embedding-001:batchEmbedContents "HTTP/1.1 200 OK"


# Store and load the index

In [37]:
# Save, it will store in a directory called storage
index.storage_context.persist()

In [ ]:
# Load
#storage_context = StorageContext.from_defaults(persist_dir='./storage')
#index = load_index_from_storage(storage_context=storage_context)

In [64]:
from llama_index.core.async_utils import asyncio_run

query_engine = index.as_query_engine()
# Python forbids creating one loop when another loop is already running
async def run():
    return await query_engine.aquery("What they eay?") 
response = await run()

2025-09-29 22:23:09,730 - INFO - AFC is enabled with max remote calls: 10.


In [65]:
from IPython.display import Markdown, display
display(Markdown(f"<b>{response}</b>"))

<b>Some dinosaurs consumed plants, others consumed meat, and some even consumed other dinosaurs.
</b>